# Title

### Amelia Zhang, Jaza Khan, Sora Takamura, Yolanda Peng

## **Introduction**

Understanding how players engage with online games can help developers and computer science researchers design better systems and keep players active on the server. The UBC team of computer science researchers, led by Frank Wood, has created a Minecraft server that records detailed behavioural data from players who have different backgrounds. These data provide a unique opportunity to study how different players interact with the game. 

Newsletter subscription can serve as an indicator of players’ interest and engagement. Therefore, identifying which types of players are most likely to subscribe and understanding how their backgrounds and in-game behaviours differ is valuable for improving recruitment strategies. 

Our research question is:

> Can a player's experience level, gender, age, and played hours predict whether a player subscribes to a game-related newsletter?

To answer this question, we are using the dataset collected from the Minecraft research server, which contains players' basic information and behavioural variables.

## **Data Description**

This dataset contains `196 observations` and `7 variables`. Each row represents data from a single player, including their personal details and information when they play Minecraft.
The purpose of this dataset is to explore what player characteristics are more predictive of newsletter subscription. 

| Variable | Type | Description | Summary / Mean Value | Possible Issues |
|----------|------|-------------|----------------------|-----------------|
| `experience`| categorical | player's level of experience. | 5 levels: Beginner, Amateur, Regular, Veteran, and Pro. Most players fall into the Amateur group. |
| `subscribe`| categorical | TRUE if the player subscribed to the newsletter; FALSE otherwise. | TRUE: 142, FALSE: 52 |
| `hashedEmail` | character | anonymized player's email address. | NA |
| `played_hours` | numerical | total hours each player has spent playing the game. | mean: 5.90 (hours) | There seem to be outliers which may skew results hence data may need to be standardized. |
| `name` | character | name | NA |
| `gender` | categorical | gender. | This dataset has six gender identities, where the majority of players are male. |
| `Age` | numerical | age. | mean: 21.14 (years) | Type may need to be changed to type Integer to optimize for memory since that will not impact data regardless. |

In this project, the **response variable** is `subscribe`, which indicates whether a player has subscribed to the newsletter or not. The **explanatory variables** are `experience`, `played_hours`, `genders`, and `Age` because these variables represent a player's characteristics and activity levels that would affect their decision to subscribe. Those irrelevant columns such as `hashedEmail` and `name` will be removed. This dataset has behavioral information for each player, which makes it possible to examine subscription patterns across different player types and activity levels.

In [2]:
library(tidyverse)
library(tidymodels)
library(repr)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

## Loading the dataset

In [4]:
players <- read_csv("https://raw.githubusercontent.com/yolandapengx/dsci-100-project-009-18/refs/heads/main/players.csv")
players

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


## Data Wrangling

In [10]:
# removing irrelevent columns
players_clean <- players |>
    select(experience,
           subscribe,
           played_hours,
           gender,
           Age) |>
# removing rows with missing values
    filter(!is.na(Age)) |>
# converting Age to integer type
    mutate(Age = as.integer(Age))

players_clean

experience,subscribe,played_hours,gender,Age
<chr>,<lgl>,<dbl>,<chr>,<int>
Pro,TRUE,30.3,Male,9
Veteran,TRUE,3.8,Male,17
Veteran,FALSE,0.0,Male,17
⋮,⋮,⋮,⋮,⋮
Veteran,FALSE,0.3,Male,22
Amateur,FALSE,0.0,Prefer not to say,57
Amateur,FALSE,2.3,Male,17


## Mean Values

In [11]:
players_mean <- players_clean |>
select(Age, played_hours) |>
summarize(Mean_Ages = mean(Age, na.rm = TRUE), Mean_Played_Hours = mean(played_hours, na.rm = TRUE))

players_mean

Mean_Ages,Mean_Played_Hours
<dbl>,<dbl>
21.13918,5.904639
